In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import geopandas as gpd
from shapely.geometry import Point, Polygon
import random
from scipy.stats import truncnorm
import sys

In [2]:
# добавляем папку modules в sys.path, чтобы импортировать функции созданные ранее

sys.path.append(os.path.abspath(os.path.join("..", "modules")))

In [3]:
# ранее созданные функции генерации времени

import antifraud_data_gen.time_generation as timegen

In [4]:
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', None)

In [5]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01\\notebooks'

In [6]:
os.chdir("..")

In [23]:
category_stats = pd.read_csv("./data/cleaned_data/category_stats.csv")
clients_with_geo = pd.read_csv("./data/cleaned_data/clients_with_geo.csv")

# В этой функции именно что select - сэмплирование. А в функции наверху - генерация массива весов для семплирования
Мб  потом джоин весов к периодам

## прототипирование выбора случайного времени

In [ ]:
timestamps = pd.DataFrame(pd.Series(pd.date_range(pd.to_datetime("2017-01-01", format="%Y-%m-%d"), \
                                                  pd.to_datetime("2024-12-31", format="%Y-%m-%d"), freq='min'), name="timestamp"))
timestamps["hour"] = timestamps.timestamp.dt.hour

In [ ]:
timestamps.head()

In [ ]:
day_time = timestamps[(timestamps.hour >= 8) & (timestamps.hour < 22)].reset_index(drop=True)
day_time

In [ ]:
    def get_time_for_trans(trans_df, client_id, round_clock_cat, is_fraud):
        # дата и время последней транзакции клиента
        last_txn_time = trans_df[trans_df.client_id == client_id].time.max()
        
        # если нет предыдущей транзакции
        if last_txn_time is np.nan:
            if round_clock_cat == 1 and not is_fraud:
                txn_time = timestamps.sample(n=1, weights=, replace=True).iloc[0]

            else:
                txn_time = day_time.sample(n=1, replace=True).iloc[0]

        # если есть предыдущая транзакция
        else:
            txn_time = timestamps.sample(n=1, replace=True).iloc[0]
            trans_time_diff = txn_time - last_txn_time
            one_hour_delta = pd.Timedelta(3600, unit="s")

            # если время между текущей и последней транзакцией меньше часа, то добавим время чтобы было час минимум
            if trans_time_diff < one_hour_delta:
                time_addition = one_hour_delta - trans_time_diff
                txn_time = txn_time + time_addition

In [ ]:
transactions_time_test = gpd.GeoDataFrame(
            {"client_id": pd.Series(dtype="int64"),
            "time": pd.Series(dtype="datetime64[ns]"),
            "amount": pd.Series(dtype="float64"),
            "type": pd.Series(dtype="string"),
            "channel": pd.Series(dtype="string"),
            "category": pd.Series(dtype="string"),
            "online":pd.Series(dtype="int64"),
            "merchant_id":pd.Series(dtype="int64"),
            "merchant_loc":pd.Series(dtype="geometry"),
            "client_ip":pd.Series(dtype="string"),
            "ip_location": pd.Series(dtype="string"),
             "account_to": pd.Series(dtype="int64"),
            "is_fraud": pd.Series(dtype="int64")})
transactions_time_test

In [17]:
# добавим колонку онлайн или не онлайн категория со значениями True и False соответсвенно

category_stats.loc[category_stats.category.str.contains("net"), "online"] = True
category_stats.loc[~category_stats.category.str.contains("net"), "online"] = False
category_stats.head(6)

,category,avg_amt,amt_std,cat_count,share,online
0,gas_transport,63.577001,15.828399,56370,0.101436,False
1,grocery_pos,115.885327,51.552330,52553,0.094568,False
2,home,57.995413,48.085281,52345,0.094193,False
3,shopping_pos,76.862457,232.484678,49791,0.089597,False
4,kids_pets,57.506913,48.748482,48692,0.087620,False
5,shopping_net,83.481653,237.219758,41779,0.075180,True


In [18]:
# добавим долю категории, ее "вес" среди категорий - для определения распростаненности категории

category_stats["share"] = category_stats.cat_count.div(category_stats.cat_count.sum())
category_stats.head()

,category,avg_amt,amt_std,cat_count,share,online
0,gas_transport,63.577001,15.828399,56370,0.101436,False
1,grocery_pos,115.885327,51.552330,52553,0.094568,False
2,home,57.995413,48.085281,52345,0.094193,False
3,shopping_pos,76.862457,232.484678,49791,0.089597,False
4,kids_pets,57.506913,48.748482,48692,0.087620,False


In [19]:
category_stats_final = category_stats.copy()

In [20]:
# умножим средние суммы транзакций по категориям и стандартное отклонение сумм на 10, для приближенности к ценам в рублях

category_stats_final[["avg_amt","amt_std"]] = category_stats_final[["avg_amt","amt_std"]] * 10
category_stats_final

,category,avg_amt,amt_std,cat_count,share,online
0,gas_transport,635.770012,158.283987,56370,0.101436,False
1,grocery_pos,1158.853270,515.523301,52553,0.094568,False
2,home,579.954129,480.852810,52345,0.094193,False
3,shopping_pos,768.624573,2324.846777,49791,0.089597,False
4,kids_pets,575.069126,487.484822,48692,0.087620,False
5,shopping_net,834.816532,2372.197581,41779,0.075180,True
6,entertainment,639.848399,642.299347,40104,0.072166,False
7,personal_care,482.330206,494.109413,39327,0.070768,False
8,food_dining,507.779383,484.490535,39268,0.070662,False
9,health_fitness,538.674317,479.652511,36674,0.065994,False


### Найдем долю фрода по категориям из датасета kaggle. Для определения вероятности фрода при генерации транзакций

In [21]:
fraud_kaggle = pd.read_csv("./data/raw_data/fraudTest.csv.zip", compression="zip")

In [22]:
fraud_trans_count_by_cat = fraud_kaggle.query("is_fraud == 1") \
                                        .groupby("category", as_index=False).agg({"trans_num":"count"}) \
                                        .rename(columns={"trans_num":"fraud_count"})
fraud_trans_count_by_cat.head()

,category,fraud_count
0,entertainment,59
1,food_dining,54
2,gas_transport,154
3,grocery_net,41
4,grocery_pos,485


In [1086]:
cat_stats_full = category_stats_final.merge(fraud_trans_count_by_cat, on="category")
cat_stats_full.head(3)

,category,avg_amt,amt_std,cat_count,online,share,fraud_count
0,gas_transport,635.770012,158.283987,56370,False,0.101436,154
1,grocery_pos,1158.853270,515.523301,52553,False,0.094568,485
2,home,579.954129,480.852810,52345,False,0.094193,67


In [1087]:
# доля фрода в категории

cat_stats_full["fraud_share"] = cat_stats_full.fraud_count.div(cat_stats_full.cat_count)
cat_stats_full.head()

,category,avg_amt,amt_std,cat_count,online,share,fraud_count,fraud_share
0,gas_transport,635.770012,158.283987,56370,False,0.101436,154,0.002732
1,grocery_pos,1158.853270,515.523301,52553,False,0.094568,485,0.009229
2,home,579.954129,480.852810,52345,False,0.094193,67,0.001280
3,shopping_pos,768.624573,2324.846777,49791,False,0.089597,213,0.004278
4,kids_pets,575.069126,487.484822,48692,False,0.087620,65,0.001335


### Добавим критерий возможности круглосуточной покупки в категории

In [1088]:
cat_stats_full["round_clock_cat"] = False

In [1089]:
round_clock = ['gas_transport', 'grocery_pos','shopping_net', 'food_dining', 'misc_pos', 'misc_net', 'grocery_net']

for category in round_clock:
    cat_stats_full.loc[cat_stats_full.category == category, "round_clock_cat"] = True
cat_stats_full

,category,avg_amt,amt_std,cat_count,online,share,fraud_count,fraud_share,round_clock_cat
0,gas_transport,635.770012,158.283987,56370,False,0.101436,154,0.002732,True
1,grocery_pos,1158.853270,515.523301,52553,False,0.094568,485,0.009229,True
2,home,579.954129,480.852810,52345,False,0.094193,67,0.001280,False
3,shopping_pos,768.624573,2324.846777,49791,False,0.089597,213,0.004278,False
4,kids_pets,575.069126,487.484822,48692,False,0.087620,65,0.001335,False
5,shopping_net,834.816532,2372.197581,41779,True,0.075180,506,0.012111,True
6,entertainment,639.848399,642.299347,40104,False,0.072166,59,0.001471,False
7,personal_care,482.330206,494.109413,39327,False,0.070768,70,0.001780,False
8,food_dining,507.779383,484.490535,39268,False,0.070662,54,0.001375,True
9,health_fitness,538.674317,479.652511,36674,False,0.065994,52,0.001418,False


### Сгенерируем id для онлайн мерчантов.
Просто, чтобы было чем заполнить поле merchant_id для онлайн покупок

In [783]:
offline_merch_max_id = int(selling_points_8_merchants.merchant_id.max())
online_merchant_ids = pd.Series([i for i in range(offline_merch_max_id + 1, offline_merch_max_id + 201)])
online_merchant_ids

0      6777
1      6778
2      6779
3      6780
4      6781
       ... 
195    6972
196    6973
197    6974
198    6975
199    6976
Length: 200, dtype: int64

# Основная функция генератор транзакций

In [1297]:
# Генерация POS транзакций
def generate_pos_transactions(client_info, start_date, end_date, categories, fraud_ips, trans_df, merchants_df \
                              ,online_merchant_ids, num=50, fraud_rate=0.05):
    """
    client_info  - датафрейм с данными клиента
    start_date - первая транзакция
    end_date - последняя транзакция
    num - количество транзакций на клиента
    fraud_rate - доля фрода во всех транзакциях
    categories - датафрейм с категориями и их характеристиками
    fraud_ips - датафрейм с фродовыми ip адресами
    trans_df - датафрейм с транзакциями.
               Куда их писать и куда обращаться за информацией по предыдущим транзакциям
    merchants_df - датафрейм с оффлайн мерачантами
    online_merchant_ids - id для онлайн мерчантов
    """
    timestamps = pd.Series(pd.date_range(pd.to_datetime(start_date), pd.to_datetime(end_date), freq='min'))
    
    day_time = timestamps[(timestamps.dt.hour >= 6) & (timestamps.dt.hour < 22)].reset_index(drop=True)
    night_time = timestamps[((timestamps.dt.hour < 6) & (timestamps.dt.hour >= 0)) \
                | (timestamps.dt.hour >= 22) & (timestamps.dt.hour <= 23)] \
                .reset_index(drop=True)

    def get_time_for_trans(trans_df, client_id, round_clock_cat, is_fraud):
        # дата и время последней транзакции клиента
        last_txn_time = trans_df[trans_df.client_id == client_id].time.max()
        
        # если нет предыдущей транзакции
        if last_txn_time is np.nan:
            if round_clock_cat == 1 and not is_fraud:
                txn_time = timestamps.sample(n=1, weights=, replace=True).iloc[0]

            else:
                txn_time = day_time.sample(n=1, replace=True).iloc[0]

        # если есть предыдущая транзакция
        else:
            txn_time = timestamps.sample(n=1, replace=True).iloc[0]
            trans_time_diff = txn_time - last_txn_time
            one_hour_delta = pd.Timedelta(3600, unit="s")

            # если время между текущей и последней транзакцией меньше часа, то добавим время чтобы было час минимум
            if trans_time_diff < one_hour_delta:
                time_addition = one_hour_delta - trans_time_diff
                txn_time = txn_time + time_addition
    
    for lab, row in client_info.iterrows():
        pos_txns = [] # сюда временно будут добавляться созданные транзакции для текущего клиента

        
        # данные клиента
        client_id = client_info.loc[lab, "client_id"]
        client_area = client_info.loc[lab, "area"]
        client_area_geometry = client_info.loc[lab, "geometry"]
        client_ip = client_info.loc[lab, "home_ip"]

        # цикл создания n транзакций для текущего клиента
        for _ in range(num):
            
                    
            # случайный выбор категории транзакции    
            category = categories.sample(1, replace=True, weights=categories.share)
            category_name = category["category"].iloc[0]
            round_clock = category["round_clock_cat"].iloc[0]
            # средняя сумма для этой категории
            amt_mean = category["avg_amt"].iloc[0]
            # стандартное отклонение сумм для этой категории
            amt_std = category["amt_std"].iloc[0]
            # случайно сгенерированная сумма не менее 1
            amount = max(1, np.random.normal(amt_mean, amt_std))

            # будет ли транзакция фродом или нет. 
            is_fraud = random.random() < fraud_rate

            # генерация данных для оффлайн покупок - не фрод
            if category["online"].iloc[0] == 0 and not is_fraud:
                merchant = merchants_df.loc[(merchants_df.area == client_area) & (merchants_df.category == category_name)].sample(1, replace=True)
                merchant_id = merchant["merchant_id"].iloc[0]
                merchant_loc = merchant["merchant_loc"].iloc[0]
                
                pos_txns.append({
                "client_id": client_id, "time": txn_time, "amount": round(amount, 2), "type": "purchase",
                "channel": "POS", "category": category_name, "online": category["online"].iloc[0],
                "merchant_id": merchant_id, "merchant_loc": merchant_loc, "is_fraud": is_fraud
            })

            trans_df_one_client = pd.DataFrame(pos_txns)
        trans_df = pd.concat([trans_df, trans_df_one_client], ignore_index=True)

    return trans_df

SyntaxError: expected argument value expression (2438120794.py, line 31)

### Создадим пустой датафрейм для записи транзакций в него

In [ ]:
transactions = gpd.GeoDataFrame(
            {"client_id": pd.Series(dtype="int64"),
            "time": pd.Series(dtype="datetime64[ns]"),
            "unix_time":pd.Series(dtype="int64"),
            "amount": pd.Series(dtype="float64"),
            "type": pd.Series(dtype="string"),
            "channel": pd.Series(dtype="string"),
            "category": pd.Series(dtype="string"),
            "online":pd.Series(dtype="int64"),
            "merchant_id":pd.Series(dtype="int64"),
            "merchant_loc":pd.Series(dtype="geometry"),
            "client_ip":pd.Series(dtype="string"),
            "ip_location": pd.Series(dtype="string"),
             "account_to": pd.Series(dtype="int64"),
            "is_fraud": pd.Series(dtype="int64")})
transactions

In [ ]:
client_subset = clients_with_geo.query("client_id == 1")

In [ ]:
client_subset

In [ ]:
trans_for_test = transactions.copy()
trans_for_test

In [ ]:
trans_gen_test_01 = generate_pos_transactions(client_info=client_subset, start_date="2017-01-01", end_date="2022-12-31", num=5, fraud_rate=0.05, \
                              categories=cat_stats_full, fraud_ips=fraud_ips, trans_df=trans_for_test, \
                              merchants_df = selling_points_8_merchants, online_merchant_ids=online_merchant_ids)

In [ ]:
trans_gen_test_01.sort_values("time")

In [1038]:
legit_merge_hour_and_total = legit_by_cat_and_hour.merge(legit_kaggle_by_cat, on="category") \
                                                [["category", "trans_hour", "trans_count", "total_trans"]] \
                                                .sort_values(["category", "trans_hour"])
legit_merge_hour_and_total.shape

In [1039]:
# legit_merge_hour_and_total["hour_share"] = legit_merge_hour_and_total.trans_count.div(legit_merge_hour_and_total.total_trans)
# legit_merge_hour_and_total.head()

In [1037]:
# fig, axes = plt.subplots(nrows=cat_stats_full.shape[0], figsize=(10, 60))

# # df = legit_merge_hour_and_total.query("category == 'health_fitness'")
# for ax, category in zip(axes, cat_stats_full["category"]):
#     sns.lineplot(data=legit_merge_hour_and_total.query("category == @category"), x="trans_hour", y="hour_share", ax=ax)
#     ax.set_title(category)
#     ax.set_ylim(0)
# plt.tight_layout()    
# plt.show()

In [1026]:
# df = legit_merge_hour_and_total.query("category == 'grocery_pos'")

# p = sns.barplot(data=df, x="trans_hour", y="trans_count")
# p.set_ylim(0);

In [909]:
# Генерация POS транзакций
def generate_pos_transactions(client_info, start_date, end_date, categories, fraud_ips, trans_df, merchants_df \
                              ,online_merchant_ids, num=50, fraud_rate=0.05):
    """
    client_info  - датафрейм с данными клиента
    start_date - первая транзакция
    end_date - последняя транзакция
    num - количество транзакций на клиента
    fraud_rate - доля фрода во всех транзакциях
    categories - датафрейм с категориями и их характеристиками
    fraud_ips - датафрейм с фродовыми ip адресами
    trans_df - датафрейм с транзакциями.
               Куда их писать и куда обращаться за информацией по предыдущим транзакциям
    merchants_df - датафрейм с оффлайн мерачантами
    online_merchant_ids - id для онлайн мерчантов
    """
    timestamps = pd.Series(pd.date_range(pd.to_datetime(start_date), pd.to_datetime(end_date), freq='min'))
    
    day_time = timestamps[(timestamps.dt.hour >= 6) & (timestamps.dt.hour < 22)].reset_index(drop=True)
    night_time = timestamps[((timestamps.dt.hour < 6) & (timestamps.dt.hour >= 0)) \
                | (timestamps.dt.hour >= 22) & (timestamps.dt.hour <= 23)] \
                .reset_index(drop=True)

    def get_time_for_trans(trans_df, client_id, round_clock_cat, is_fraud):
        # дата и время последней транзакции клиента
        last_txn_time = trans_df[trans_df.client_id == client_id].time.max()
        
        # если нет предыдущей транзакции
        if last_txn_time is np.nan:
            if round_clock_cat == 1 and not is_fraud:
                txn_time = timestamps.sample(n=1, weights=, replace=True).iloc[0]

            else:
                txn_time = day_time.sample(n=1, replace=True).iloc[0]

        # если есть предыдущая транзакция
        else:
            txn_time = timestamps.sample(n=1, replace=True).iloc[0]
            trans_time_diff = txn_time - last_txn_time
            one_hour_delta = pd.Timedelta(3600, unit="s")

            # если время между текущей и последней транзакцией меньше часа, то добавим время чтобы было час минимум
            if trans_time_diff < one_hour_delta:
                time_addition = one_hour_delta - trans_time_diff
                txn_time = txn_time + time_addition
    
    for lab, row in client_info.iterrows():
        pos_txns = [] # сюда временно будут добавляться созданные транзакции для текущего клиента

        
        # данные клиента
        client_id = client_info.loc[lab, "client_id"]
        client_area = client_info.loc[lab, "area"]
        client_area_geometry = client_info.loc[lab, "geometry"]
        client_ip = client_info.loc[lab, "home_ip"]

        # цикл создания n транзакций для текущего клиента
        for _ in range(num):
            
                    
            # случайный выбор категории транзакции    
            category = categories.sample(1, replace=True, weights=categories.share)
            category_name = category["category"].iloc[0]
            round_clock = category["round_clock_cat"].iloc[0]
            # средняя сумма для этой категории
            amt_mean = category["avg_amt"].iloc[0]
            # стандартное отклонение сумм для этой категории
            amt_std = category["amt_std"].iloc[0]
            # случайно сгенерированная сумма не менее 1
            amount = max(1, np.random.normal(amt_mean, amt_std))

            # будет ли транзакция фродом или нет. 
            is_fraud = random.random() < fraud_rate

            # генерация данных для оффлайн покупок - не фрод
            if category["online"].iloc[0] == 0 and not is_fraud:
                merchant = merchants_df.loc[(merchants_df.area == client_area) & (merchants_df.category == category_name)].sample(1, replace=True)
                merchant_id = merchant["merchant_id"].iloc[0]
                merchant_loc = merchant["merchant_loc"].iloc[0]
                
                pos_txns.append({
                "client_id": client_id, "time": txn_time, "amount": round(amount, 2), "type": "purchase",
                "channel": "POS", "category": category_name, "online": category["online"].iloc[0],
                "merchant_id": merchant_id, "merchant_loc": merchant_loc, "is_fraud": is_fraud
            })

            trans_df_one_client = pd.DataFrame(pos_txns)
        trans_df = pd.concat([trans_df, trans_df_one_client], ignore_index=True)

    return trans_df

In [872]:
client_subset = clients_with_geo.query("client_id == 1")

In [907]:
client_subset

,client_id,district_id,birth_date,sex,district_code,region,area,timezone,lat,lon,population,clients,geometry,home_ip
0,1,18,1970-12-13,female,18,Рязанская,Рязань,UTC+3,54.625457,39.735999,525062,60,"POLYGON ((39.53667 54.6385, 39.53677 54.63889,...",2.60.0.1


In [910]:
trans_for_test = transactions.copy()
trans_for_test

,client_id,time,amount,type,channel,category,online,merchant_id,merchant_loc,client_ip,ip_location,account_to,is_fraud


In [911]:
trans_gen_test_01 = generate_pos_transactions(client_info=client_subset, start_date="2017-01-01", end_date="2022-12-31", num=5, fraud_rate=0.05, \
                              categories=cat_stats_full, fraud_ips=fraud_ips, trans_df=trans_for_test, \
                              merchants_df = selling_points_8_merchants, online_merchant_ids=online_merchant_ids)

In [912]:
trans_gen_test_01.sort_values("time")

,client_id,time,amount,type,channel,category,online,merchant_id,merchant_loc,client_ip,ip_location,account_to,is_fraud
0,1,2020-06-17 15:32:00,388.33,purchase,POS,misc_pos,0.0,2876.0,POINT (39.58864244042683 54.64360405430491),<NA>,<NA>,NaN,0
1,1,2021-07-04 23:52:00,539.23,purchase,POS,personal_care,0.0,3729.0,POINT (39.74591808527166 54.637572015694126),<NA>,<NA>,NaN,0
2,1,2021-11-30 08:10:00,105.37,purchase,POS,kids_pets,0.0,6262.0,POINT (39.79949759003459 54.60411155098016),<NA>,<NA>,NaN,0


In [ ]:
# amounts = []
# category = cat_stats_full.sample(1, replace=True, weights=cat_stats_full.share)
# for _ in range(100):

    
#     amt_mean = category["avg_amt"].iloc[0]
#     amt_std = category["amt_std"].iloc[0]
#     amount = max(1, np.random.normal(amt_mean, amt_std))

#     amounts.append(amount)
#     # print(f"""
#     # Category: {category["category"]}
#     # Amount: {amount}
#     # """)

# print(f"""Category: {category["category"]}""")
# pd.Series(amounts).mean()

In [ ]:
# cat_stats_full.query("category == @category['category'].iloc[0]")

### Часть от оригинальной функции от GPT

In [908]:
                
            # генерация данных для онлайн покупок: ip
            # if category["online"] == 1:
                
            # Гео — нормальный город
            # city = random.choice(list(city_coords.keys()))
            # lat, lon = city_coords[city]
            # lat += np.random.normal(0, 0.01)
            # lon += np.random.normal(0, 0.01)
    
            
            # if is_fraud:
            #     # Фрод: другой город, высокая сумма, необычная категория
            #     category = "luxury"
            #     amount = np.random.uniform(20000, 150000)
            #     lat += np.random.normal(5, 2)  # далеко
            #     lon += np.random.normal(5, 2)
    
            # pos_txns.append({
            #     "client_id": client_id,
            #     "date": txn_date,
            #     "amount": round(amount, 2),
            #     "type": "purchase",
            #     "channel": "POS",
            #     "category": category,
            #     "latitude": lat,
            #     "longitude": lon,
            #     "is_fraud": is_fraud
            # })

    # return pd.DataFrame(pos_txns)

# Внизу образец генерации транзакций от GPT

In [ ]:
import random
from datetime import timedelta

# Категории и примерные средние суммы (можно расширить)
categories = {
    "grocery": (300, 100),
    "electronics": (10000, 5000),
    "transport": (150, 50),
    "restaurants": (800, 200),
    "clothing": (2000, 800)
}

# Список городов с координатами
city_coords = {
    "Москва": (55.75, 37.62),
    "Санкт-Петербург": (59.94, 30.31),
    "Казань": (55.79, 49.12),
    "Екатеринбург": (56.84, 60.61)
}

In [ ]:
# Генерация POS транзакций
def generate_pos_transactions(account_id, client_id, start_date, end_date, num=50, fraud_rate=0.05):
    dates = pd.date_range(start_date, end_date, freq='D').to_pydatetime().tolist()
    pos_txns = []

    for _ in range(num):
        txn_date = random.choice(dates)
        category = random.choice(list(categories.keys()))
        mean, std = categories[category]
        amount = max(1, np.random.normal(mean, std))

        # Гео — нормальный город
        city = random.choice(list(city_coords.keys()))
        lat, lon = city_coords[city]
        lat += np.random.normal(0, 0.01)
        lon += np.random.normal(0, 0.01)

        is_fraud = random.random() < fraud_rate
        if is_fraud:
            # Фрод: другой город, высокая сумма, необычная категория
            category = "luxury"
            amount = np.random.uniform(20000, 150000)
            lat += np.random.normal(5, 2)  # далеко
            lon += np.random.normal(5, 2)

        pos_txns.append({
            "account_id": account_id,
            "client_id": client_id,
            "date": txn_date,
            "amount": round(amount, 2),
            "type": "purchase",
            "channel": "POS",
            "category": category,
            "latitude": lat,
            "longitude": lon,
            "is_fraud": is_fraud
        })

    return pd.DataFrame(pos_txns)

## Нормальное распределение

In [707]:
# 56.310481, 43.989214

In [708]:
# 56.335365, 43.922722 # std


In [709]:
# 56.335365 - 56.304909

In [710]:
# 43.989214 - 43.922722

In [711]:
# from shapely.geometry import Point, Polygon

# def Random_Points_in_Bounds_Normal(polygon, number, mean_x, mean_y, std_x, std_y):   
    
#     x = np.random.normal(mean_x, std_x, number)
#     y = np.random.normal(mean_y, std_y, number)
#     return x, y

In [712]:
# nn_poly = district_ru_merge.query("area == 'Нижний Новгород'").iloc[0,-1]

In [713]:
# gdf_poly = gpd.GeoDataFrame(index=["myPoly"], geometry=[nn_poly])
# gdf_poly

In [714]:
# x,y = Random_Points_in_Bounds_Normal(nn_poly, 1000, mean_x=43.989214, mean_y= 56.310481, std_x=0.066491999, std_y=0.030456)
# df = pd.DataFrame()
# df['points'] = list(zip(x,y))
# df['points'] = df['points'].apply(Point)
# gdf_points = gpd.GeoDataFrame(df, geometry='points')

In [715]:
# Sjoin = gpd.tools.sjoin(gdf_points, gdf_poly, predicate="within", how='left')

# # Keep points in "myPoly"
# pnts_in_poly = gdf_points[Sjoin.index_right=='myPoly']

# # Plot result
# base = gdf_poly.boundary.plot(linewidth=1, edgecolor="black")
# gdf_points.plot(ax=base, linewidth=1, color="red", markersize=8)
# plt.show()

In [716]:
ru_cities_tent_subset.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [717]:
from shapely.ops import transform
from pyproj import Geod

# Две координаты: (долгота, широта)
coord1 = (37.6173, 55.7558)   # Москва
coord2 = (65.502079, 44.848393)   # Кызылорда 2300 km

# Геодезический расчёт по эллипсоиду WGS84
geod = Geod(ellps="WGS84")
distance_m = geod.inv(coord1[0], coord1[1], coord2[0], coord2[1])[-1]

print(f"Расстояние: {distance_m / 1000:.2f} км")

Расстояние: 2304.08 км


In [718]:
# files = os.listdir()
# files
# name_part = "10_kalin_ru"

# for file in files:
#     file_split =  file.split(".")
#     if file_split[0] == "gis_osm_places_a_free_1":
#         new_name = name_part + "." + file_split[-1]
#         print(new_name)
#         os.rename(file, new_name)
    # elif file_split[0] == "07_ural_ru":
    #     new_name = name_part + "." + file_split[-1]
    #     print(new_name)
    #     os.rename(file, new_name)
        
# os.rename("test.txt", "renamed")
# "asdasd.shp".split(".")[-1]

In [719]:
# os.chdir("./data/raw_data/geo/")

In [720]:
# gpd.read_file("data/raw_data/geo/01_central_ru.shp")

In [721]:
# # ru_city_shapes = 
# pd.concat([gpd.read_file(f"./data/raw_data/geo/{file}").query("fclass == 'city'") for file in geo_files])

In [722]:
# os.chdir("C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01\\")